In [1]:
configs = { 
"fs.azure.account.auth.type": "CustomAccessToken",
"fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}

In [2]:
print("Note: Yo, dawg! Gold zone disabled for now, re-enable after we get it prod zone")

In [3]:
filesystems = ['bronze', 'silver'] #filesystems = ['bronze', 'silver'] #
lakeName = 'prodanalytics1'
subDirectory = ''

In [4]:
#dbutils.fs.unmount('/mnt/%s' % (filesystems[0]))
#dbutils.fs.unmount('/mnt/%s' % (filesystems[1]))
#dbutils.fs.unmount('/mnt/%s' % (filesystems[2]))


In [5]:
def Diff(li1, li2): 
    return (list(set(li1) - set(li2)))
def makefilter(mntName):
   def iterator(x):
      return x.mountPoint == '/mnt/%s' % (mntName) 
   return iterator
def getMountedZones():
  mounted = []
  for mnt in filesystems:
    result = list(filter(makefilter(mnt), dbutils.fs.mounts()))
    if (len(result) == 1):
        mounted.append(mnt)
  return mounted  
def mount(filesystem):
  dbutils.fs.mount(
    source = "abfss://%s@%s.dfs.core.windows.net/%s" % (filesystem, lakeName, subDirectory),
    mount_point = "/mnt/%s" % (filesystem),
    extra_configs = configs)
  print('Mounted /mnt/%s' % (filesystem))

#Diff(filesystems, getMountedZones())


In [6]:
zonesToMnt = Diff(filesystems, getMountedZones())

if (len(zonesToMnt) == 0):
  print('All Zones already mounted')
else:
  for zone in zonesToMnt:
    mount(zone)
  



All Zones already mounted

In [7]:
dbutils.fs.mounts()

Out[11]: [MountInfo(mountPoint='/mnt/Test_bronze', source='abfss://bronze@prodanalytics1.dfs.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/mnt/silver', source='abfss://silver@prodanalytics1.dfs.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/mnt/bronze', source='abfss://bronze@prodanalytics1.dfs.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType='')]

In [8]:
#display(dbutils.fs.ls("/mnt/") )


path,name,size
dbfs:/mnt/Test_bronze/,Test_bronze/,0
dbfs:/mnt/bronze/,bronze/,0
dbfs:/mnt/silver/,silver/,0


In [9]:
dbutils.fs.ls('./mnt/bronze/cab')

Out[13]: [FileInfo(path='dbfs:/mnt/bronze/cab/billing/', name='billing/', size=0),
 FileInfo(path='dbfs:/mnt/bronze/cab/cabmanager/', name='cabmanager/', size=0),
 FileInfo(path='dbfs:/mnt/bronze/cab/operational/', name='operational/', size=0)]